<a href="https://colab.research.google.com/github/jmishra01/AutomateLightGBMModel/blob/main/Assignment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Download dataset from Kaggle into Colab

In [ ]:
!pip install -q kaggle

In [ ]:
from google.colab import files

In [ ]:
!mkdir ~/.kaggle

In [ ]:
files.upload()

In [ ]:
!cp kaggle.json ~/.kaggle/

In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d paultimothymooney/stock-market-data

 99% 1.02G/1.03G [00:07<00:00, 164MB/s]
100% 1.03G/1.03G [00:07<00:00, 144MB/s]


In [ ]:
!unzip stock-market-data.zip

Archive:  stock-market-data.zip
replace stock_market_data/forbes2000/csv/A.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
!ls 

kaggle.json  sample_data  stock_market_data  stock-market-data.zip


In [ ]:
!cd stock_market_data

In [ ]:
!apt install tree

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following NEW packages will be installed:
  tree
0 upgraded, 1 newly installed, 0 to remove and 20 not upgraded.
Need to get 40.7 kB of archives.
After this operation, 105 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tree amd64 1.7.0-5 [40.7 kB]
Fetched 40.7 kB in 0s (113 kB/s)
Selecting previously unselected package tree.
(Reading database ... 159447 files and directories currently installed.)
Preparing to unpack .../tree_1.7.0-5_amd64.deb ...
Unpacking tree (1.7.0-5) ...
Setting up tree (1.7.0-5) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...


In [ ]:
!ls stock_market_data/forbes2000/csv/ | head

AAALY.csv
AACAY.csv
AAL.csv
AAP.csv
AAPL.csv
ABBV.csv
ABC.csv
ABLT.csv
ABT.csv
ABTZY.csv


In [ ]:
!tree -d

.
├── sample_data
└── stock_market_data
    ├── forbes2000
    │   ├── csv
    │   └── json
    ├── nasdaq
    │   ├── csv
    │   └── json
    ├── nyse
    │   ├── csv
    │   └── json
    └── sp500
        ├── csv
        └── json

14 directories


In [ ]:
!pip install git+https://github.com/DynamicsAndNeuralSystems/pycatch22.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/DynamicsAndNeuralSystems/pycatch22.git to /tmp/pip-req-build-cz21yidd
  Running command git clone -q https://github.com/DynamicsAndNeuralSystems/pycatch22.git /tmp/pip-req-build-cz21yidd
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for UNKNOWN: filename=UNKNOWN-0.0.0-cp37-cp37m-linux_x86_64.whl size=105346 sha256=f4419b166c09095252417dcf49478fb369e527d867f1b12f73aaf4c3b020c86d
  Stored in directory: /tmp/pip-ephem-wheel-cache-y0zmz0tg/wheels/26/91/41/35eeff61307b4f088746c582a99c4e0f2cf9a19471326bf561
Successfully built UNKNOWN


In [ ]:
import os

In [ ]:
import pandas as pd

In [ ]:
import pycatch22

In [ ]:
from pathlib import Path

In [ ]:
df = pd.read_csv("./stock_market_data/forbes2000/csv/AAALY.csv")

In [ ]:
df

,Date,Low,Open,Volume,High,Close,Adjusted Close
0,15-03-2013,23.290001,23.290001,100,23.290001,23.290001,16.989510
1,18-03-2013,23.290001,23.290001,0,23.290001,23.290001,16.989510
2,19-03-2013,23.290001,23.290001,0,23.290001,23.290001,16.989510
3,20-03-2013,23.290001,23.290001,0,23.290001,23.290001,16.989510
4,21-03-2013,23.290001,23.290001,0,23.290001,23.290001,16.989510
...,...,...,...,...,...,...,...
2376,22-08-2022,35.299999,35.299999,0,35.299999,35.299999,35.299999
2377,23-08-2022,35.299999,35.299999,0,35.299999,35.299999,35.299999
2378,24-08-2022,35.299999,35.299999,0,35.299999,35.299999,35.299999
2379,25-08-2022,35.299999,35.299999,0,35.299999,35.299999,35.299999


In [1]:
def multiple_rolling(df,
                     windows = [1,2],
                     functions=["mean","std"],
                     columns=None,
                     ticker="ticker"):
  windows = [a for a in windows]
  if not columns:
    columns = df.columns.to_list()
  rolling_dfs = (df.groupby(df[ticker])[columns].rolling(i)                                    # 1. Create window
                  .agg(functions)                                # 1. Aggregate
                  .rename({col: '{0}_{1:d}'.format(col, i)
                                for col in columns}, axis=1).reset_index(drop=True)  # 2. Rename columns
                for i in windows)                                # For each window
  df_out = pd.concat((df, *rolling_dfs), axis=1)
  da = df_out.iloc[:,len(df.columns):]
  da = [col[0] + "_" + col[1] for col in  da.columns.to_list()]
  df_out.columns = df.columns.to_list() + da 
  return  df_out  